In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import random
import pandas as pd
sp.init_printing()

In [ ]:
N = 21
time = np.arange(0, 40, 1)
InitialCondition = [N - 1, 1, 0]

beta = 0.7/N
gamma = 0.2

def ODE(time, y):
    Spos, Ipos, Rpos = y
    return [-beta*Ipos*Spos, beta*Ipos*Spos - gamma*Ipos, gamma*Ipos]



sol = solve_ivp(ODE, (0,40), [N - 1, 1, 0], t_eval = time)

In [ ]:
sol

In [ ]:
Fig_Data = plt.figure()

plt.plot(sol['t'], sol['y'][0], label = 'S')
plt.plot(sol['t'], sol['y'][1], label = 'I')
plt.plot(sol['t'], sol['y'][2], label = 'R')

plt.title('', size= 20)
plt.ylabel('', size= 20)
plt.xlabel('', size= 20)
plt.legend(fontsize= 12, loc = 'upper right')
plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
plt.grid()

plt.show()